## Hierarchical Time Series Forecasting

Hierarchical Time Series Forecasting is a method of forecasting that deals with time series data organized in a hierarchy, where the data is structured at multiple levels of aggregation.

It means you have data that is organized in levels — like a family tree, but for numbers over time.

For example, imagine you want to predict sales:

* First, total sales for the whole company (top level)

* Then sales for each region (middle level)

* Then sales for each store in those regions (bottom level)

The important part is that the smaller parts add up to the bigger parts. So, if you add all the stores’ sales, it should equal the sales of the whole region, and all regions together should equal the total company sales.

When you make forecasts, you want these numbers to make sense together — that’s called **coherence**.

### Key Benefits

Hierarchical time series forecasting offers several key benefits that enhance accuracy, consistency, and decision-making across multiple levels of data.


1. **Structured Data** – Real-world data is naturally organized in hierarchical levels.
2. **Coherent Forecasts** – Ensures forecasts at all levels add up correctly.
3. **Better Decisions** – Supports planning for all roles, from top-level to local.
4. **Higher Accuracy** – Uses information from all levels to improve predictions.
5. **Flexible Analysis** – Allows forecasting at any level of detail needed.


In [ ]:
# Install statsmodels if not already present your device.
# !pip install statsmodels

### 0.1 Dataset Setup

Here we are just creating a simple dataframe based on the below strucutre.

In [ ]:
# @title Data Structure
print("""
Total
├── Region_1
│   ├── Store_A1
│   └── Store_A2
└── Region_2
    ├── Store_B1
    └── Store_B2
""")



Total
├── Region_1
│   ├── Store_A1
│   └── Store_A2
└── Region_2
    ├── Store_B1
    └── Store_B2



In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")



# Simulated monthly data for 4 stores across 2 regions over 5 months
data = pd.DataFrame({
    'Month': pd.date_range(start='2025-01-01', periods=5, freq='ME'),

    # Region 1 stores
    'Store_A1': [100, 120, 130, 125, 140],
    'Store_A2': [80, 85, 90, 95, 100],

    # Region 2 stores
    'Store_B1': [70, 75, 80, 85, 90],
    'Store_B2': [60, 65, 70, 75, 80],
})

# Calculate Region sums
data['Region_1'] = data['Store_A1'] + data['Store_A2']
data['Region_2'] = data['Store_B1'] + data['Store_B2']

# Calculate Total across all regions
data['Total'] = data['Region_1'] + data['Region_2']

data


,Month,Store_A1,Store_A2,Store_B1,Store_B2,Region_1,Region_2,Total
0,2025-01-31,100,80,70,60,180,130,310
1,2025-02-28,120,85,75,65,205,140,345
2,2025-03-31,130,90,80,70,220,150,370
3,2025-04-30,125,95,85,75,220,160,380
4,2025-05-31,140,100,90,80,240,170,410


### 0.2 Model Setup

Here we are just using a simple model called ARIMA to forecast the next value based on the given input row data.

In [ ]:
def forecast_next(series):
    model = ARIMA(series, order=(1, 1, 0))
    model_fit = model.fit()
    return model_fit.forecast().iloc[0]  # Forecast the next time step


### Methods of Hierarchical Time Series Forecasting

#### 1. Bottom-Up Approach

Forecasts are made at the lowest level (e.g., store in our case), and then aggregated upwards to get forecasts for higher levels (e.g., region, total).

Best for: When low-level data is reliable.

In [ ]:
# Approach Style: Store -> Region -> Total (Bottom-Up)

# Forecasting the next value for each individual store using the ARIMA model
forecast_store_a1 = forecast_next(data['Store_A1'])  # Forecast for Store A1
forecast_store_a2 = forecast_next(data['Store_A2'])  # Forecast for Store A2
forecast_store_b1 = forecast_next(data['Store_B1'])  # Forecast for Store B1
forecast_store_b2 = forecast_next(data['Store_B2'])  # Forecast for Store B2

# --- Bottom-Up Approach ---
# Combine store forecasts to get region-level forecasts
bu_region1 = forecast_store_a1 + forecast_store_a2  # Region 1 = Store A1 + A2
bu_region2 = forecast_store_b1 + forecast_store_b2  # Region 2 = Store B1 + B2

# Add region-level forecasts to get the total forecast
bu_total = bu_region1 + bu_region2  # Total = Region 1 + Region 2

# Store all the forecasts in a dictionary
forecast_results_bottomUp = {
    "Store_A1": round(forecast_store_a1, 2),
    "Store_A2": round(forecast_store_a2, 2),
    "Store_B1": round(forecast_store_b1, 2),
    "Store_B2": round(forecast_store_b2, 2),
    "Region_1": round(bu_region1, 2),
    "Region_2": round(bu_region2, 2),
    "Total": round(bu_total, 2),
}

# Convert the forecast dictionary to a DataFrame for cleaner display
forecast_results_bottomUp = pd.DataFrame.from_dict(forecast_results_bottomUp, orient='index', columns=['Forecast'])

# Show the resulting forecast table
forecast_results_bottomUp


,Forecast
Store_A1,143.57
Store_A2,105.00
Store_B1,95.00
Store_B2,85.00
Region_1,248.57
Region_2,180.00
Total,428.57


#### 2. Top-Down Approach

Forecasts are made at the highest aggregate level (e.g., total sales), then disaggregated down to lower levels (e.g., region, store) using historical proportions or ratios.

Best for: When aggregate data is more accurate or reliable than lower-level data.

In [ ]:
# --- Top-Down Approach ---
# Approach Style
# Total -> Region -> Store

# Step 1: Forecast total directly
forecast_total = forecast_next(data['Total'])

# Step 2: Calculate historical shares
# Based on last available data
total_last = data['Total'].iloc[-1]
region1_last = data['Region_1'].iloc[-1]
region2_last = data['Region_2'].iloc[-1]

share_region1 = region1_last / total_last
share_region2 = region2_last / total_last

share_a1 = data['Store_A1'].iloc[-1] / region1_last
share_a2 = data['Store_A2'].iloc[-1] / region1_last

share_b1 = data['Store_B1'].iloc[-1] / region2_last
share_b2 = data['Store_B2'].iloc[-1] / region2_last

# Step 3: Split total forecast to regions
td_region1 = forecast_total * share_region1
td_region2 = forecast_total * share_region2

# Step 4: Split regional forecasts to individual stores
td_store_a1 = td_region1 * share_a1
td_store_a2 = td_region1 * share_a2
td_store_b1 = td_region2 * share_b1
td_store_b2 = td_region2 * share_b2

# Step 5: Prepare forecast dictionary
forecast_results_topDown = {
    "Total": round(forecast_total, 2),
    "Region_1": round(td_region1, 2),
    "Region_2": round(td_region2, 2),
    "Store_A1": round(td_store_a1, 2),
    "Store_A2": round(td_store_a2, 2),
    "Store_B1": round(td_store_b1, 2),
    "Store_B2": round(td_store_b2, 2),
}

# Convert to DataFrame for pretty printing
forecast_results_topDown = pd.DataFrame.from_dict(forecast_results_topDown, orient='index', columns=['Forecast'])
forecast_results_topDown


,Forecast
Total,435.97
Region_1,255.20
Region_2,180.77
Store_A1,148.87
Store_A2,106.33
Store_B1,95.70
Store_B2,85.07


In the Top-Down approach, we start by forecasting the total sales for all stores combined. This overall forecast is then divided into smaller parts—first into regional forecasts **based on each region’s past contribution to the total**, and then into individual store forecasts using each store’s historical share within its region. This method flows from the top of the hierarchy (Total) down to the bottom (Stores), ensuring consistency while relying on historical proportions to guide the distribution.


#### 3. Middle-Out Approach

Forecasting is done at an intermediate level (e.g., region), then aggregated upwards and disaggregated downwards to other levels in the hierarchy.

Best for: When intermediate-level data is most reliable, balancing detail and aggregation.

In [ ]:
# --- Middle-Out Approach ---
# Approach Style
# Region -> Store and Region -> Total

# Step 1: Forecast region-level data
forecast_region1 = forecast_next(data['Region_1'])
forecast_region2 = forecast_next(data['Region_2'])

# Step 2: Calculate historical shares for stores within each region
# Based on last available data
region1_last = data['Region_1'].iloc[-1]
region2_last = data['Region_2'].iloc[-1]

share_a1 = data['Store_A1'].iloc[-1] / region1_last
share_a2 = data['Store_A2'].iloc[-1] / region1_last

share_b1 = data['Store_B1'].iloc[-1] / region2_last
share_b2 = data['Store_B2'].iloc[-1] / region2_last

# Step 3: Allocate regional forecasts to stores
mo_store_a1 = forecast_region1 * share_a1
mo_store_a2 = forecast_region1 * share_a2

mo_store_b1 = forecast_region2 * share_b1
mo_store_b2 = forecast_region2 * share_b2

# Step 4: Compute total from region forecasts
mo_total = forecast_region1 + forecast_region2

# Step 5: Create forecast result dictionary
forecast_results_middleOut = {
    "Region_1": round(forecast_region1, 2),
    "Region_2": round(forecast_region2, 2),
    "Store_A1": round(mo_store_a1, 2),
    "Store_A2": round(mo_store_a2, 2),
    "Store_B1": round(mo_store_b1, 2),
    "Store_B2": round(mo_store_b2, 2),
    "Total": round(mo_total, 2),
}

# Convert to DataFrame for neat display
forecast_results_middleOut = pd.DataFrame.from_dict(forecast_results_middleOut, orient='index', columns=['Forecast'])
forecast_results_middleOut


,Forecast
Region_1,252.77
Region_2,180.00
Store_A1,147.45
Store_A2,105.32
Store_B1,95.29
Store_B2,84.71
Total,432.77


In the Middle-Out approach, forecasting starts at the **middle level**—typically the region level—rather than the top (total) or bottom (store). We first forecast the sales for each region using historical data. Then, we **break down each region's forecast into store-level forecasts** using the past contribution of each store within that region. Finally, we **add up the region forecasts** to get the total forecast. This method balances detail and manageability by using regional forecasts as a base and spreading the information both upward to total and downward to stores.


#### 4. Optimal Reconciliation

Forecasts are independently made at all levels, then mathematically adjusted to ensure coherence across the hierarchy while minimizing overall forecast errors.

Best for: When forecasts are available at all levels and you want statistically optimal, consistent results.



In hierarchical time series forecasting, we often generate forecasts at all levels of a hierarchy (e.g., national, regional, store), but the forecasts may not add up properly. This violates a property called coherence.

Optimal reconciliation ensures that forecasts are coherent (i.e., higher levels equal the sum of lower levels). It Minimize forecast error variance


**Mathematical Setup**

Variables

* Let the base forecasts (unreconciled) for all levels be $$ \hat{\mathbf{y}}_t \in \mathbb{R}^n $$

* Let the reconciled forecasts (coherent) be $$ \tilde{\mathbf{y}}_t \in \mathbb{R}^n $$


* Let the summing matrix, mapping bottom-level forecasts to all level be  $$ \mathbf{S} \in \mathbb{R}^{n \times m} $$


* Let the reconciled bottom-level forecasts be $$ \hat{\mathbf{b}}_t \in \mathbb{R}^m $$

Then:
$$
\tilde{\mathbf{y}}_t = \mathbf{S} \hat{\mathbf{b}}_t
$$

Goal
Minimize the mean squared error of the reconciled forecasts:
$$
\min_{\hat{\mathbf{b}}_t} \mathbb{E} \left[ \left| \tilde{\mathbf{y}}_t - \mathbf{y}_t \right|^2 \right] \quad \text{subject to} \quad \tilde{\mathbf{y}}_t = \mathbf{S} \hat{\mathbf{b}}_t
$$

Optimal Reconciliation Solution
Using the forecast error covariance matrix (W), the solution is:
$$
\hat{\mathbf{b}}_t = (\mathbf{S}^\top \mathbf{W}^{-1} \mathbf{S})^{-1} \mathbf{S}^\top \mathbf{W}^{-1} \hat{\mathbf{y}}_t
$$
And hence,
$$
\tilde{\mathbf{y}}_t = \mathbf{S} \hat{\mathbf{b}}_t
$$


**Comparison to Other Methods**

$$
\begin{array}{|l|c|c|c|}
\hline
\textbf{Method} & \textbf{Coherent?} & \textbf{Uses Error Covariance?} & \textbf{Optimal?} \\
\hline
\text{Bottom-up} & \checkmark & \times & \times \\
\text{Top-down} & \checkmark & \times & \times \\
\text{Optimal Reconciliation} & \checkmark & \checkmark & \checkmark \\
\hline
\end{array}
$$



**Summary**

* Optimal reconciliation adjusts base forecasts to be coherent.

* Uses knowledge of forecast error covariances.

* Statistically minimizes total forecast error.



##### Imports

Since the procedure for **Optimal Reconciliation** involves complex matrix operations and error covariance estimation, we will be using a library to simplify the implementation.

The [`hierarchicalforecast`](https://nixtla.github.io/hierarchicalforecast/) library provides a user-friendly and efficient way to perform various hierarchical forecasting methods, including:

* **Bottom-Up**
* **Top-Down**
* **Middle-Out**
* **Optimal Reconciliation** (e.g., MinTrace)

This library also includes useful utilities for working with hierarchical structures and ensures that forecasts are **coherent** across different levels of the hierarchy.By using `hierarchicalforecast`, we can focus more on analyzing results and less on implementing the underlying mathematical operations from scratch.


In [ ]:
from hierarchicalforecast.utils import aggregate
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

##### Creating Dataframe

We are generating a monthly sales dataset for four stores grouped under two regions to simulate a hierarchical time series structure. This synthetic dataset allows us to practice and test reconciliation techniques such as bottom-up, top-down, and optimal reconciliation in a controlled, simplified environment.

Each store has 5 months of sales data, starting from January 2025 to May 2025 (month-end). The dataset is structured in a **long format** with columns for region, store, ds (timestamp), and y (sales value).

In [ ]:
# Define 5 monthly dates starting January 2025 (month-end)
ds = pd.date_range(start='2025-01-01', periods=5, freq='ME')

# Define the hierarchy with the exact values provided
hierarchy = [
    ('Region1', 'Store_A1', [100, 120, 130, 125, 140]),
    ('Region1', 'Store_A2', [80, 85, 90, 95, 100]),
    ('Region2', 'Store_B1', [70, 75, 80, 85, 90]),
    ('Region2', 'Store_B2', [60, 65, 70, 75, 80]),
]

# Generate the dataset
rows = []
for region, store, values in hierarchy:
    for t in range(5):
        rows.append({
            'region': region,
            'store': store,
            'ds': ds[t],
            'y': values[t]
        })
custom_df = pd.DataFrame(rows)

custom_df

,region,store,ds,y
0,Region1,Store_A1,2025-01-31,100
1,Region1,Store_A1,2025-02-28,120
2,Region1,Store_A1,2025-03-31,130
3,Region1,Store_A1,2025-04-30,125
4,Region1,Store_A1,2025-05-31,140
5,Region1,Store_A2,2025-01-31,80
6,Region1,Store_A2,2025-02-28,85
7,Region1,Store_A2,2025-03-31,90
8,Region1,Store_A2,2025-04-30,95
9,Region1,Store_A2,2025-05-31,100


##### Setting up the dataframe for forecasting



The `spec` defines the structure of the hierarchy in the dataset. It is a list of lists, where each inner list represents a level in the hierarchy.

For example, `['region']` defines the **first level** (regional totals), and `['region', 'store']` defines the **bottom level** (individual store-level data).
This specification helps the forecasting library understand how the data is organized so that it can correctly apply hierarchical aggregation and reconciliation techniques.


`aggregate()` – Hierarchical Aggregation Utility

The `aggregate()` function from the `hierarchicalforecast.utils` module takes the raw input DataFrame (`custom_df`) and the hierarchy specification (`spec`) to automatically compute:

* `Y_df`: A wide-format DataFrame of time series for each node in the hierarchy.
* `S`: The **summing matrix**, which defines how bottom-level nodes aggregate up to higher levels.
* `tags`: A dictionary labeling which series belong to which hierarchical level.

This function is essential for transforming long-format raw data into a structure suitable for hierarchical forecasting and reconciliation.


Here,

* **Long Format:** Each row is an observation (e.g., one store, one date, one value).


$$
\begin{array}{|c|c|c|c|}
\hline
\textbf{region} & \textbf{store} & \textbf{ds} & \textbf{y} \\
\hline
\text{Region1} & \text{Store_A1} & 2025\text{-}01\text{-}31 & 100 \\
\text{Region1} & \text{Store_A2} & 2025\text{-}02\text{-}28 & 120 \\
\hline
\end{array}
$$


* **Wide Format:** Each column is a time series (one per store or region).

$$
\begin{array}{|c|c|c|}
\hline
\textbf{ds} & \textbf{Region1/Store_A1} \\
\hline
2025\text{-}01\text{-}31 & 100 \\
2025\text{-}02\text{-}28 & 120 \\
\hline
\end{array}
$$



In [ ]:
# Define the hierarchy specification
spec = [['region'], ['region', 'store']]

# Aggregate the data to create Y_df, S, and tags
Y_df, S, tags = aggregate(custom_df, spec)

Y_df.head()

,unique_id,ds,y
0,Region1,2025-01-31,180
1,Region1,2025-02-28,205
2,Region1,2025-03-31,220
3,Region1,2025-04-30,220
4,Region1,2025-05-31,240


##### Basic Conversion and Train Test Split

In [ ]:
# Ensure 'ds' is datetime
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

# Split train/test sets
# Use January to April for training, May for testing
Y_test_df = Y_df.groupby('unique_id').tail(1)  # May 31, 2025
Y_train_df = Y_df.drop(Y_test_df.index)  # January to April 2025

##### Computing Forecasts

This line sets up and runs forecasts for the next four months for each series in `Y_train_df` by using two methods—AutoARIMA (which automatically finds and fits a suitable seasonal model) and Naive (which simply repeats the last known value). By specifying `freq='M'`, it knows to treat the data as monthly, and `n_jobs=-1` makes it compute all series in parallel for speed. The result (`Y_hat_df`) holds these “base” predictions, which you’ll later adjust (reconcile) so that forecasts at different hierarchy levels (e.g., stores vs. regions) add up correctly.


In [ ]:
fcst = StatsForecast(models=[AutoARIMA(season_length=4), Naive()],
                     freq='M', n_jobs=-1)
Y_hat_df = fcst.forecast(df=Y_train_df, h=2)  # h=2 to cover May and June

# Step 6: Filter for June 2025 forecast
Y_hat_df = Y_hat_df[Y_hat_df['ds'] == '2025-06-30']

#####  Set up hierarchical reconciliation

This code defines how to adjust (“reconcile”) your raw forecasts so they add up correctly across stores and regions. First, you list the methods you want: **BottomUp** (sum individual store forecasts up to regions), **TopDown** with “forecast\_proportions” (start from a total and split it among stores based on forecast shares), and **MiddleOut** at the “region” level (reconcile at the region first, then propagate up and down using proportions). You pass these into a `HierarchicalReconciliation` object, then call `reconcile`, supplying the base forecasts (`Y_hat_df`), the original training data (`Y_train_df`), the summing matrix `S`, and the `tags` that define the hierarchy. The output `Y_rec_df` contains adjusted forecasts that are coherent across all hierarchy levels.

In [ ]:
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='region', top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

# Step 8: Reconcile the base predictions
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, S=S, tags=tags)

In [ ]:
Y_rec_df

,unique_id,ds,AutoARIMA,Naive,AutoARIMA/BottomUp,Naive/BottomUp,AutoARIMA/TopDown_method-forecast_proportions,Naive/TopDown_method-forecast_proportions,AutoARIMA/MiddleOut_middle_level-region_top_down_method-forecast_proportions,Naive/MiddleOut_middle_level-region_top_down_method-forecast_proportions
0,Region1,2025-06-30,207.691360,220.0,209.470337,220.0,207.691360,220.0,207.691360,220.0
1,Region2,2025-06-30,157.151352,160.0,155.892227,160.0,157.151352,160.0,157.151352,160.0
2,Region1/Store_A1,2025-06-30,116.524223,125.0,116.524223,125.0,115.534614,125.0,115.534614,125.0
3,Region1/Store_A2,2025-06-30,92.946114,95.0,92.946114,95.0,92.156747,95.0,92.156747,95.0
4,Region2/Store_B1,2025-06-30,82.946114,85.0,82.946114,85.0,83.616060,85.0,83.616060,85.0
5,Region2/Store_B2,2025-06-30,72.946114,75.0,72.946114,75.0,73.535292,75.0,73.535292,75.0


Here we have different forecast based on different algorithms and approch with HierarchicalReconciliation. Since we dont have huge data the data does not vary much.

#### 4. Comparision

Here, we are comparing different forecasting outcomes to understand how each approach yields varying results, even though they all aim to predict the same values.

In [ ]:
# @title Generate Optimal Forecast DataFrame with Aggregated Regional Forecast
forecast_results_optimal = Y_rec_df[['Naive/MiddleOut_middle_level-region_top_down_method-forecast_proportions']].copy()
forecast_results_optimal.columns = ['Forecast']

forecast_results_optimal['Name'] = [
    'Region_1',
    'Region_2',
    'Store_A1',
    'Store_A2',
    'Store_B1',
    'Store_B2'
]

forecast_results_optimal.set_index('Name', inplace=True)

total_row = pd.DataFrame({
    'Forecast': [forecast_results_optimal.loc[['Region_1', 'Region_2'], 'Forecast'].sum()]
}, index=['Total'])

forecast_results_optimal = pd.concat([forecast_results_optimal, total_row])

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'bottomUp_forecast': forecast_results_bottomUp['Forecast'],
    'topDown_forecast': forecast_results_topDown['Forecast'],
    'middleOut_forecast': forecast_results_middleOut['Forecast'],
    'optimal_forecast': forecast_results_optimal['Forecast']
  # Extract 1D array from result_df
})

# Display the result
comparison_df

,bottomUp_forecast,topDown_forecast,middleOut_forecast,optimal_forecast
Region_1,248.57,255.20,252.77,220.0
Region_2,180.00,180.77,180.00,160.0
Store_A1,143.57,148.87,147.45,125.0
Store_A2,105.00,106.33,105.32,95.0
Store_B1,95.00,95.70,95.29,85.0
Store_B2,85.00,85.07,84.71,75.0
Total,428.57,435.97,432.77,380.0
